In [26]:
import matplotlib.pyplot as plt
import cv2 as cv
import os

In [30]:
folder1 = '/kaggle/input/preprocessed-ct-scans-for-covid19/Original CT Scans/nCT'
folder2 = '/kaggle/input/preprocessed-ct-scans-for-covid19/Original CT Scans/pCT'
positive = []
negative = []
for filename in os.listdir(folder2):
    img_path = os.path.join(folder2, filename)
    img = cv.imread(img_path)
    positive.append(img)
for filename in os.listdir(folder1):
    img_path = os.path.join(folder1, filename)
    img = cv.imread(img_path)
    negative.append(img)
positive = np.array(positive)
negative = np.array(negative)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4001,) + inhomogeneous part.

In [34]:
len(positive)

4001